Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [48]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce
from tqdm.auto import tqdm
from queue import PriorityQueue

In [49]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [50]:
problem_dim=1000
num_sets=1000
fitness_counter=0
x = make_set_covering_problem(problem_dim, num_sets, .3)




In [51]:
def get_set(index):
    return x[[index],:].toarray()[0]

# find how many false it takes
def distance(state):         
    return problem_dim - sum(state)


# compute logical or between all the set taken in the current state
def total_state(state):     
    return reduce(
        np.logical_or,
        [get_set(i) for i in state],
        np.array([False for _ in range(problem_dim)]),
    )


In [52]:

def fitness(state):
    cost = state.sum(axis=0)
    global fitness_counter
    fitness_counter= fitness_counter+1
    valid = np.all(
        reduce(
            np.logical_or,
            [x[[i],:].toarray()[0] for i, t in enumerate(state) if t],
            np.array([False for _ in range(problem_dim)]),
        )
    )
    return valid, cost

def random_state(state):
    new_state = copy(state)
    index = randint(0, num_sets-1)
    new_state=([index], get_set(index))
    return new_state

def key_funct(e, state):
    e_state=total_state([e])

    overlap = sum(np.logical_and(e_state , total_state(state)))
  
    return overlap+distance(e_state)

def tweak(state):
    sets_list= set(range(num_sets))-set(state[0])
    candidates = sorted(sets_list, key=lambda e:key_funct(e, state[0]) )
    new_state = state[0].__add__([candidates[0]])
    return (new_state, total_state(new_state))

actual_best_sol = None
T_start=10000
fitness_counter=0
n_run = 200

first_ind= randint(0, num_sets-1)
current_state=([first_ind], get_set(first_ind))

with tqdm(total=n_run) as pbar:
    for run in range(n_run):
        d=1
    
        new_state = random_state(current_state)
        step = 1
        T=T_start
        new_state_eval= fitness(new_state[1])
        current_state_eval = fitness(current_state[1])
        while step<=10 and (new_state_eval >= current_state_eval or np.exp(-(current_state_eval[1]-new_state_eval[1])/T)>random()) :
            if T==0:
                print("ho freddo, temperatura a zero!!")
                break
            else:
                current_state = new_state
                new_state = tweak(new_state)
                step+=1
                d = distance(new_state[1])

                if d==0 or (actual_best_sol is not None and len(new_state[0])>=len(actual_best_sol)):
                    break
                current_state_eval = new_state_eval
                new_state_eval= fitness(new_state[1])   
                T=1/np.log(1+step)*T
        if d==0:
            if actual_best_sol is None or len(new_state[0])<len(actual_best_sol):
                actual_best_sol = new_state[0]
                print(len(actual_best_sol), fitness_counter)
                ''
            current_state=(new_state[0][0], get_set(new_state[0][0]))


        pbar.update(1)  
    print(f"solution found in {fitness_counter} calls,solution has length {len(actual_best_sol)}")

  0%|          | 0/200 [00:00<?, ?it/s]

10 10
9 1939
solution found in 1993 calls,solution has length 9
